# Initialization:

In [2]:
import torch
import dqc
import dqc.xc
import dqc.utils

from DQCAdapter import DQCAdapter

In [3]:
class MyLDAX(dqc.xc.CustomXC):
    def __init__(self, a, p):
        super().__init__()
        self.a = a
        self.p = p

    @property
    def family(self):
        # 1 for LDA, 2 for GGA, 4 for MGGA
        return 1

    # NB: it is not \epsilon, it is \epsilon * \rho! 
    # It can be seen in 292-294 of dqc/hamiltonian/hcgto.py
    # It means, that V_{XC} = dget_edensityxc/drho without second term.
    def get_edensityxc(self, densinfo):
        # densinfo has up and down components
        if isinstance(densinfo, dqc.utils.SpinParam):
            # spin-scaling of the exchange energy
            return 0.5 * (self.get_edensityxc(densinfo.u * 2) + self.get_edensityxc(densinfo.d * 2))
        else:
            rho = densinfo.value.abs() + 1e-15  # safeguarding from nan
            return self.a * rho ** self.p
    
    # it is for tests
    def get_vxc_analytical(self, densinfo):
        if isinstance(densinfo, dqc.utils.SpinParam):
            # spin-scaling of the exchange energy
            return 0.5 * (self.get_vxc_analytical(densinfo.u * 2) + self.get_vxc_analytical(densinfo.d * 2))
        else:
            rho = densinfo.value.abs() + 1e-15  # safeguarding from nan
            return (self.a * self.p * rho ** (self.p - 1))
        
    def get_edensityxc_derivative(self, densinfo, number_of_parameter):
        # densinfo has up and down components
        if isinstance(densinfo, dqc.utils.SpinParam):
            # spin-scaling of the exchange energy
            return 0.5 * (self.get_edensityxc_derivative(densinfo.u * 2, number_of_parameter) 
                          + self.get_edensityxc_derivative(densinfo.d * 2, number_of_parameter))
        else:
            rho = densinfo.value.abs() + 1e-15  # safeguarding from nan
            if number_of_parameter == 0: # parameter a
                return rho ** self.p
            elif number_of_parameter == 1: # parameter p
                # TODO: it is incorrect derivative
                return self.a * rho ** (self.p - 1)

In [4]:
a = torch.nn.Parameter(torch.tensor(1.0, dtype=torch.double))
p = torch.nn.Parameter(torch.tensor(2.0, dtype=torch.double))
myxc = MyLDAX(a, p)

In [5]:
mol = dqc.Mol(moldesc="N -1 0 0; N 1 0 0", basis="3-21G")
qc = dqc.KS(mol, xc=myxc).run()
ene = qc.energy()
print(ene)

tensor(-54.0932, dtype=torch.float64, grad_fn=<AddBackward0>)


/home/radium/miniconda3/lib/python3.9/site-packages/xitorch/_impls/optimize/root/rootsolver.py:163: ConvergenceWarning: The rootfinder does not converge after 50 iterations. Best |dx|=2.508e-02, |f|=2.288e-01 at iter 32
  warnings.warn(ConvergenceWarning(msg))


In [6]:
adapter = DQCAdapter(qc)

# Check orthonormality of basis set in DQC

In [7]:
number_of_all_orbitals = adapter.get_number_of_all_orbitals()
overlap = adapter.get_overlap_matrix()

In [8]:
torch.linalg.matrix_norm(overlap - torch.eye(number_of_all_orbitals))

tensor(2.3030e-14, dtype=torch.float64)

# Check density matrix vs orbital coefficients consistency

In [9]:
dm = adapter.get_density_matrix()

In [10]:
coefficients = adapter.get_orbital_coefficients()
coefficients.size()

torch.Size([18, 7])

They must have the same norm (it can differs in two times because of $f_a=2$ in the case of RKS)

In [11]:
print(torch.linalg.matrix_norm(torch.matmul(coefficients, coefficients.t())))
print(torch.linalg.matrix_norm(dm))

tensor(2.6458, dtype=torch.float64, grad_fn=<CopyBackwards>)
tensor(5.2915, dtype=torch.float64)


In [12]:
# TODO: other checks?
# TODO: strange value of 
print(torch.linalg.matrix_norm(dm - torch.matmul(coefficients, coefficients.t())))

tensor(3.2924, dtype=torch.float64, grad_fn=<CopyBackwards>)


# Check symmetry of electron-electron repulsion tensor

$$G_{ijkl} = \iint b_i(\vec{r}) \frac{b_k(\vec{r'})b_l(\vec{r'})}{|\vec{r}-\vec{r'}|} b_j(\vec{r})d\vec{r'}d\vec{r}$$
It must have symmetry of i<->j and k<->l permutations, let's check it. We are going to calculate something like norm of remainder.

In [13]:
electronic_repulsion_tensor = adapter.get_four_center_elrep_tensor()

i<->j and k<->l should be almost zero

In [14]:
print(torch.linalg.matrix_norm(torch.linalg.matrix_norm(electronic_repulsion_tensor - electronic_repulsion_tensor.transpose(0, 1))))
print(torch.linalg.matrix_norm(torch.linalg.matrix_norm(electronic_repulsion_tensor - electronic_repulsion_tensor.transpose(2, 3))))

tensor(1.2990e-13, dtype=torch.float64)
tensor(7.9542e-15, dtype=torch.float64)


i<->k, i<->l, j<->k and j<->l should not be zero

In [15]:
print(torch.linalg.matrix_norm(torch.linalg.matrix_norm(electronic_repulsion_tensor - electronic_repulsion_tensor.transpose(0, 2))))
print(torch.linalg.matrix_norm(torch.linalg.matrix_norm(electronic_repulsion_tensor - electronic_repulsion_tensor.transpose(0, 3))))
print(torch.linalg.matrix_norm(torch.linalg.matrix_norm(electronic_repulsion_tensor - electronic_repulsion_tensor.transpose(1, 2))))
print(torch.linalg.matrix_norm(torch.linalg.matrix_norm(electronic_repulsion_tensor - electronic_repulsion_tensor.transpose(1, 3))))

tensor(13.9926, dtype=torch.float64)
tensor(13.9926, dtype=torch.float64)
tensor(13.9926, dtype=torch.float64)
tensor(13.9926, dtype=torch.float64)


# Check density at grid sums to number of electrons

In [24]:
density_at_grid = qc.get_system().get_hamiltonian()._dm2densinfo(dm)
volumes_at_grid = qc.get_system().get_hamiltonian().grid.get_dvolume()
number_of_electrons_integration_of_density = torch.einsum("i,i->", density_at_grid.value, volumes_at_grid)
number_of_electrons_from_occupancies = adapter.get_orbital_occupancy().sum()

In [25]:
number_of_electrons_integration_of_density - number_of_electrons_from_occupancies

tensor(-8.0931e-05, dtype=torch.float64)

# Compare analytical and autograd $V_{XC}[\rho]$

Let's talk about LDA. For it:
$$V_{XC}[\rho](\vec{r}) = \epsilon_{xc}[\rho](\vec{r}) + \frac{\partial \epsilon_{xc}[\rho](\vec{r})}{\partial \rho(\vec{r})}$$

But DQC uses autograd.grad instead of analytical expression.

In [16]:
from dqc.utils.datastruct import ValGrad, SpinParam


vxc_autograd_at_grid = qc.get_system().get_hamiltonian().xc.get_vxc(density_at_grid)
vxc_autograd_matrix = qc.get_system().get_hamiltonian()._get_vxc_from_potinfo(vxc_autograd_at_grid)
# eqiuvalent to: vxc_autograd_matrix = qc.get_system().get_hamiltonian().get_vxc(dm)

In [17]:
vxc_analytical_at_grid = ValGrad(qc.get_system().get_hamiltonian().xc.get_vxc_analytical(density_at_grid))
vxc_analytical_matrix = qc.get_system().get_hamiltonian()._get_vxc_from_potinfo(vxc_analytical_at_grid)

In [18]:
print(torch.linalg.vector_norm(vxc_autograd_at_grid.value - vxc_analytical_at_grid.value))
print(torch.linalg.matrix_norm(vxc_autograd_matrix.fullmatrix() - vxc_analytical_matrix.fullmatrix()))

tensor(2.6382e-14, dtype=torch.float64, grad_fn=<LinalgVectorNormBackward0>)
tensor(0., dtype=torch.float64, grad_fn=<CopyBackwards>)
